**Pytorch keypoints RCNN**

**17 keypoints:**[ ‘nose’, ‘left_eye’, ‘right_eye’, ‘left_ear’, ‘right_ear’, ‘left_shoulder’, ‘right_shoulder’, ‘left_elbow’, ‘right_elbow’, ‘left_wrist’, ‘right_wrist’, ‘left_hip’, ‘right_hip’, ‘left_knee’, ‘right_knee’, ‘left_ankle’, ‘right_ankle’]

**edges =** [
    (0, 1), (0, 2), (2, 4), (1, 3), (6, 8), (8, 10),
    (5, 7), (7, 9), (5, 11), (11, 13), (13, 15), (6, 12),
    (12, 14), (14, 16), (5, 6)
]

PyTorch pre-trained models for human pose and keypoint detection


**Import Libs**

In [28]:
import torch
from google.colab import drive
import numpy as np
import cv2
import argparse
from PIL import Image
from torchvision.transforms import transforms as transforms
import torchvision
import os
import matplotlib


In [19]:
def get_model(min_size=800):
    # initialize the model
    model = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=True,
                                                                   num_keypoints=17,
                                                                   min_size=min_size)

    return model

In [20]:
# Load the model onto the computation device and set it to eval mode
# Set the computation device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = get_model().to(device).eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=KeypointRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=KeypointRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
drive_image_dir ="/content/drive/MyDrive/imgs_test_set"
output_dir = "/content/drive/MyDrive/imgs_test_set/2D_pose/pytorch_rcnn"

In [23]:
# Define the transform to convert the image to a tensor
transform = transforms.Compose([
    transforms.ToTensor()
])

In [27]:
edges = [
    (0, 1), (0, 2), (2, 4), (1, 3), (6, 8), (8, 10),
    (5, 7), (7, 9), (5, 11), (11, 13), (13, 15), (6, 12),
    (12, 14), (14, 16), (5, 6)
]

In [29]:
def draw_keypoints_and_boxes(outputs, image):
    for output in outputs:
        # Get the detected keypoints, bounding boxes, and confidence scores
        keypoints = output['keypoints']
        boxes = output['boxes']
        scores = output['scores']

        # Iterate through instances
        for i in range(len(keypoints)):
            if scores[i] > 0.9:  # Proceed if confidence is above 0.9
                keypoints_array = keypoints[i].cpu().detach().numpy()
                keypoints_array = keypoints_array[:, :].reshape(-1, 3)
                for p in range(keypoints_array.shape[0]):
                    # Draw the keypoints
                    cv2.circle(image, (int(keypoints_array[p, 0]), int(keypoints_array[p, 1])),
                                3, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

                # Draw the lines joining the keypoints
                for ie, e in enumerate(edges):
                    # Get different colors for the edges
                    rgb = matplotlib.colors.hsv_to_rgb([
                        ie / float(len(edges)), 1.0, 1.0
                    ])
                    rgb = rgb * 255
                    # Join the keypoint pairs to draw the skeletal structure
                    cv2.line(image, (int(keypoints_array[e[0], 0]), int(keypoints_array[e[0], 1])),
                             (int(keypoints_array[e[1], 0]), int(keypoints_array[e[1], 1])),
                             tuple(rgb), 2, lineType=cv2.LINE_AA)

                # Draw the bounding boxes around the objects
                box = boxes[i].cpu().detach().numpy()
                cv2.rectangle(image, (int(box[0]), int(box[1]), int(box[2]), int(box[3])),
                              color=(0, 255, 0),
                              thickness=2)

    return image

In [30]:
# Loop through all images in the input directory
for filename in os.listdir(drive_image_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(drive_image_dir, filename)

        image = Image.open(image_path).convert('RGB')
        # NumPy copy of the image for OpenCV functions
        orig_numpy = np.array(image, dtype=np.float32)
        # Convert the NumPy image to OpenCV BGR format
        orig_numpy = cv2.cvtColor(orig_numpy, cv2.COLOR_RGB2BGR) / 255.
        # Transform the image
        image = transform(image)
        # Add a batch dimension
        image = image.unsqueeze(0).to(device)

        # Get the detections, forward pass the image through the model
        with torch.no_grad():
            outputs = model(image)

        # Draw the keypoints, lines, and bounding boxes
        output_image = draw_keypoints_and_boxes(outputs, orig_numpy)

        # Set the save path for the output image
        save_path = os.path.join(output_dir, filename)
        cv2.imwrite(save_path, output_image * 255.)